Simple demo showing cosine similarity 

Reuirements:
* [huggingface token](https://huggingface.co/docs/hub/en/security-tokens) (free)
  * either add the hf token to your colab secret keys or
* colab account with some (free) credits available


In [1]:
# un comment !pip below for the first run...
# !pip install  rich sentence_transformers torch

In [1]:
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
# import os
# from pathlib import Path

**NOTE:** may switch to TPU coming up _(nvidia rtx 4070 on home dev machine)_

In [3]:
mycuda = torch.cuda.is_available()
print(f"Cuda available?: {mycuda}")

Cuda available?: True


**Note**: add (free) hugging face token to `secrets manager` (see key icon on left menue)

In [4]:
import os, getpass

# uncomment the next two lines if not using secrets (untested in colab, but works on my local dev...)
if "HF_API_TOKEN" not in os.environ:
    os.environ["HF_API_TOKEN"] = getpass.getpass("Enter Hugging Face token:")

In [5]:
from rich.highlighter import RegexHighlighter
from rich.table import Table
from rich.theme import Theme
from rich.console import Console
from rich.text import Text
from rich.padding import Padding
from rich import print  # https://rich.readthedocs.io/en/stable/markup.html#console-markup

embedder model is small, fast, free

may want to experiment with other embedder models

In [6]:
model = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(model)
print("embedder model loaded: {}".format(model))

embedder model loaded: all-MiniLM-L6-v2

Next cell has been adjusted from github repo to get sample text files (already extracted). Comments near bottom show initial filesystem read

Note: this is a hackish comparison of the same pdf/same-content extracted in different ways. Things look fairly similar, but tika "seemed" to do better with fonts, or something. Python had strange breaks in words.

In [8]:
# sentences = ['
git_files = ['https://raw.githubusercontent.com/seanoc5/wooly-tongue/main/data/ces-wp-24-16.pypdf.txt',
             'https://raw.githubusercontent.com/seanoc5/wooly-tongue/main/data/ces-wp-24-16.tika.txt']


# Issue request: r => requests.models.Response
documents = []
for gf in git_files:
    r = requests.get(gf)
    txt = r.text
    fname = gf.split("/")[-1]
    doc = Document(content=txt, meta={'file_name':fname})
    documents.append(doc)

# data_dir = Path("../../data")
# filename_fn = lambda filename: {'file_name': os.path.basename(filename)}
# documents = SimpleDirectoryReader(data_dir, filename_as_id=True, file_metadata=filename_fn).load_data(show_progress=True)
print(f"documents loaded: {len(documents)}")


NameError: name 'requests' is not defined

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


Using langchain text_spliter, but this could be just about anything.  


*   spacy
*   nltk
*   haystack
*   many others....

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 256,
    chunk_overlap  = 20,
)

Some semi-curated inputs to experiment with

**NOTE**: this version uses `cosine-similarity and torch.topk to find the highest 5 scores`  
I should have variations on this actual search/matching with various other approaches RSN...

In [ ]:
queries = [
    "what segments show decline in adoption of AI or LLM.",
    "automation trends in business"
    "Measuring AI in US economy.",
    "Large language models",
]

Note: package `rich` has some nice (new to me) functionality.  
This could be overkill, but I got it working, so keeping it in the demo workbook.  
With a bit of editing/refactoring we should be able to various interesting terms (hand-crafted synonym sets) to highlight the semantic-ness of these approaches

In [ ]:
hl_list = [
        r"(?i)(?P<learn>ai|learn[a-z]*|lesson|ml|train[a-z]*|recogni[a-z]*|analy[a-z]*)",
        r"(?i)(?P<machine>(machine|computer|bot|agent|system|recommenda|speech|voice)[a-z]*)",
        r"(?i)(?P<model>embed[a-z]*|large|language|model[a-z]*|ml|llm|generat[a-z]*)",
        r"(?i)(?P<orgs>firm[a-z]*|org[a-z]*|compan[a-z]*|instit[a-z]*)",
        r"(?i)(?P<actions>(chat|search|build|generate|follow|create|use)[a-z]*)",
    ]

class MyHighlighter(RegexHighlighter):
    """Apply style to anything that looks interesting..."""
    base_style = "example."
    highlights = hl_list

my_ml = MyHighlighter()

theme = Theme(
    {"example.learn": "bold magenta",
     "example.machine": "bold green",
     "example.model": "bold blue",
     "example.orgs": "bold yellow",
     "example.actions": "bold red",
     })
console = Console(highlighter=my_ml, theme=theme)
# print a test message to show different colors for different `named capture group`s above
console.print("Sample highlighting of ML models to use for  computer machine learning (maybe chat??) lessons and training with embeddings in your company or organization")


Sample highlighting of ML models to use for  computer machine learning (maybe chat??) lessons and training with 
embeddings in your company or organization

In [ ]:
for doc in documents:
    # print(f"document: {doc}")
    fname = doc.meta['file_name']
    txt = doc.content
    docs = text_splitter.split_text(txt)
    docs_embeddings = embedder.encode(docs, convert_to_tensor=True, show_progress_bar=True)

    # playing with rich console output... nothing more than eye candy
    print(f"[bold red]================== File: {fname} docs split: {len(docs)} ================== [/bold red]")

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(5, len(docs))
    for query in queries:
        print(f"\n[bold blue]----------------------------\nQuery: {query} [/bold blue]")
        # todo -- cache query embeddings, reuse in subsequent txt file comparisons
        query_embedding = embedder.encode(query, convert_to_tensor=True)

        table = Table(title=f"File: {fname}\nquery:{query}", show_lines=True, highlight=True)
        table.add_column("node id", style="blue", )     #no_wrap=True
        table.add_column("Score", style="blue", )     #no_wrap=True
        # table.add_column("Tika Score",  style="green")
        table.add_column("Tika Text")


        # We use cosine-similarity and torch.topk to find the highest 5 scores
        cos_scores = util.cos_sim(query_embedding, docs_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)

        # tika_cos_scores = util.cos_sim(query_embedding, tika_embeddings)[0]
        # tika_top_results = torch.topk(tika_cos_scores, k=top_k)


        for score, idx in zip(top_results[0], top_results[1]):
            t = docs[idx]
            node_id = idx.item()
            my_score = f"{score.item():.4f}"        # hacked approach to rounding to 4 decimals
            # table.add_row(docs[idx], "(Score: {:.4f})".format(score), 'tika score', 'tika text')
            table.add_row(str(node_id), str(my_score), t)

        """
        # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
        hits = hits[0]      #Get the hits for the first query
        for hit in hits:
            print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
        """

        console.print(table)


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

================== File: ces-wp-24-16.pypdf.txt docs split: 653 ================== 

----------------------------
Query: what segments show decline in adoption of AI or LLM. 

                                           File: ces-wp-24-16.pypdf.txt                                            
                            query:what segments show decline in adoption of AI or LLM.                             
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node id ┃ Score  ┃ Tika Text                                                                                    ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 351     │ 0.6237 │ reason for no AI use for the period 2016–2018 was also, by a large margin, the               │
│         │        │ inapplicability of                                                                           │
│         │        │                                                                                              │
│         │        │ AI to a business. AI being too expensive and not mature were the next most common factors    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 138     │ 0.6209 │ us understand why businesses have not adopted AI. It asks for reasons why the business does  │
│         │        │ not                                                                                          │
│         │        │                                                                                              │
│         │        │ plan to use AI during the next six months and provides 10 possible reasons and then “Other.” │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 16      │ 0.6078 │ most common reason for non-adoption is the inapplicability of AI to the business.            │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 343     │ 0.6063 │ designed to uncover some of the key impediments to AI adoption or use.                       │
│         │        │                                                                                              │
│         │        │ Table 7 shows that the most common reason, cited by 80.9% of the businesses not expecting    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 390     │ 0.5922 │ 6.  While many businesses use AI to replace worker tasks and existing equipment/software,    │
│         │        │                                                                                              │
│         │        │ there is little evidence that AI use is associated with a decline in firm employment. In     │
└─────────┴────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

----------------------------
Query: automation trends in businessMeasuring AI in US economy. 

...

                                            File: ces-wp-24-16.tika.txt                                            
                            query:what segments show decline in adoption of AI or LLM.                             
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node id ┃ Score  ┃ Tika Text                                                                                    ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 16      │ 0.6244 │ most common reason for non-adoption is the inapplicability of AI to the business.            │
│         │        │                                                                                              │
│         │        │                                                                                              │
│         │        │ 0F                                                                                           │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 351     │ 0.6237 │ reason for no AI use for the period 2016–2018 was also, by a large margin, the               │
│         │        │ inapplicability of                                                                           │
│         │        │                                                                                              │
│         │        │ AI to a business. AI being too expensive and not mature were the next most common factors    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 138     │ 0.6209 │ us understand why businesses have not adopted AI. It asks for reasons why the business does  │
│         │        │ not                                                                                          │
│         │        │                                                                                              │
│         │        │ plan to use AI during the next six months and provides 10 possible reasons and then “Other.” │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 343     │ 0.6063 │ designed to uncover some of the key impediments to AI adoption or use.                       │
│         │        │                                                                                              │
│         │        │ Table 7 shows that the most common reason, cited by 80.9% of the businesses not expecting    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 390     │ 0.5922 │ 6. While many businesses use AI to replace worker tasks and existing equipment/software,     │
│         │        │                                                                                              │
│         │        │ there is little evidence that AI use is associated with a decline in firm employment. In     │
└─────────┴────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

----------------------------
Query: automation trends in businessMeasuring AI in US economy. 

                                            File: ces-wp-24-16.tika.txt                                            
                          query:automation trends in businessMeasuring AI in US economy.                           
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node id ┃ Score  ┃ Tika Text                                                                                    ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 23      │ 0.7365 │ been ongoing for several years and investigating the impact of AI on the economy is an       │
│         │        │ active                                                                                       │
│         │        │                                                                                              │
│         │        │ area of research. However, real time, comprehensive data on AI use by businesses has been    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 405     │ 0.7346 │ some time with rapid advances in recent years, AI use by businesses remains in relatively    │
│         │        │ early                                                                                        │
│         │        │                                                                                              │
│         │        │ days (see Chapter 7 of the Economic Report of President (2024) and Agrawal, Gans and         │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 378     │ 0.7206 │ We analyze recently collected high-frequency data on firms’ use of AI in producing goods     │
│         │        │                                                                                              │
│         │        │ and services based on the Business Trends and Outlook Survey. Several key facts emerge:      │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 138     │ 0.7169 │ us understand why businesses have not adopted AI. It asks for reasons why the business does  │
│         │        │ not                                                                                          │
│         │        │                                                                                              │
│         │        │ plan to use AI during the next six months and provides 10 possible reasons and then “Other.” │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 60      │ 0.7138 │ Looking deeper, we see that businesses that use AI typically utilize it for marketing        │
│         │        │                                                                                              │
│         │        │ automation, virtual agents and chatbots, natural language processing and data/text           │
│         │        │ analytics. On                                                                                │
└─────────┴────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

----------------------------
Query: Large language models 

                                            File: ces-wp-24-16.tika.txt                                            
                                            query:Large language models                                            
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node id ┃ Score  ┃ Tika Text                                                                                    ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 247     │ 0.4407 │ small firms compared to the earlier patterns from the ABS is the accelerating use of         │
│         │        │ generative                                                                                   │
│         │        │                                                                                              │
│         │        │ AI particularly among these firms following the introduction of Large Language Models (LLMs) │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 623     │ 0.4199 │ Select all that apply.                                                                       │
│         │        │ • Machine learning                                                                           │
│         │        │ • Natural language processing                                                                │
│         │        │ • Virtual agents or chat bots                                                                │
│         │        │ • Speech/voice recognition using AI                                                          │
│         │        │ • Recommendation systems based on AI                                                         │
│         │        │ • Large language models                                                                      │
│         │        │ • Text analytics using AI                                                                    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 275     │ 0.3919 │ recognition (15.9%). These are followed by an undefined “other” category (15.9%), machine    │
│         │        │                                                                                              │
│         │        │ learning (13.6%) and large language models (11.4%). The relatively low use rates for the     │
│         │        │ latter                                                                                       │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 278     │ 0.3918 │ instance, marketing automation typically uses both machine learning and large language       │
│         │        │ models,                                                                                      │
│         │        │                                                                                              │
│         │        │ typically embedded in virtual agents such as ChatGPT that can generate customized marketing  │
│         │        │                                                                                              │
│         │        │ messages.                                                                                    │
├─────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ 427     │ 0.3828 │ Eloundou, Tyna, Sam Manning, Pamela Mishkin, and Daniel Rock (2023). “GPTs are GPTs: An      │
│         │        │ Early Look at the Labor Market Impact Potential of Large Language Models,” Working           │
│         │        │ Paper, OpenAI.                                                                               │
│         │        │                                    

In [ ]:
print("Done!")

Done!